In [34]:
using LinearAlgebra

In [35]:
#make sure to cite this package
using SPICE

In [36]:
using Downloads: download 

In [37]:
const LSK = "https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls"

"https://naif.jpl.nasa.gov/pub/naif/generic_kernels/lsk/naif0012.tls"

In [38]:
const SPK = "https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de440.bsp"

"https://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de440.bsp"

In [39]:
# Download kernels
#Time Kernel
#download(LSK, "naif0012.tls")

#JPL planetary and lunar ephermis de440
#Contains the orbit of all the planets
# from years 1549 to 2650 

#download(SPK, "de440.bsp")

In [40]:
# Load leap seconds kernel
furnsh("naif0012.tls")

In [41]:
# Convert the calendar date to ephemeris seconds past J2000
et = utc2et("2018-02-06T20:45:00")

5.712219691849319e8

In [42]:
# Load a planetary ephemeris kernel
furnsh("de440.bsp")

In [102]:
#gravitational parameters for the bodies (km3/s2)

#moon 
μ_m = 4902.80058214776

#earth
μ_e = 398600.432896939

#sun
μ_s = 132712440017.987

1.32712440017987e11

In [103]:
#units used to normalize

#distance
l_star = 385692.50

#time
t_star = 377084.152667038

#in km
lunar_radius = 1738.2 

1738.2

In [104]:
function ephemeris_model_EarthMoon(x, t)

    println("in")
    xdot = zeros(6)

    xdot[1:3] = x[4:6]

    #position of the spacecraft relative to the moon 
    r_qi = x[1:3]


    time = et + t

    rqj_earth = spkpos("earth", time, "J2000", "none", "moon")[1]
    rqj_sun = spkpos("sun", time, "J2000", "none", "moon")[1]
    
    rij_earth = rqj_earth- r_qi 
    rij_sun = rqj_sun-r_qi

    #r̈_qi - vectors that define the position of spacecraft (P_i)
    #relative to the central body (in this case the moon)

    #\ splits up into multiple lines
    xdot[4:6] = (-μ_m/(norm(r_qi))^3)*r_qi + 
                μ_e.*((rij_earth/norm(rij_earth)^3)-(rqj_earth/norm(rqj_earth)^3)) + 
                μ_s.*((rij_sun/norm(rij_sun)^3)-(rqj_sun/norm(rqj_sun)^3));

    return xdot

end

ephemeris_model_EarthMoon (generic function with 1 method)

In [105]:
xtest = [1,2,3,4,5,6]
xdot = ephemeris_model_EarthMoon(xtest, 0)

in


[-1.596243945293849e-13, 1.5498034957138158e-14, -8.12755285609417e-14]


6-element Vector{Float64}:
    4.0
    5.0
    6.0
  -93.59489803095211
 -187.18979606195023
 -280.78469409296196

In [44]:
# Get the position of Mars at `et` w.r.t. Earth

#arguments of this function
#spkpos(targ, et, ref, abcorr, obs)
#targ: target body name
#et: observer epoch
#ref: reference frame of output position vector
#abcorr: Aberration correction flag
#obs: Observing body name

#test = spkpos("mars_barycenter", et, "J2000", "none", "earth")

([-9.561252750118901e7, -2.045743839170729e8, -8.601309297738348e7], 806.0297773638883)

In [45]:
test[1]

3-element Vector{Float64}:
 -9.561252750118901e7
 -2.045743839170729e8
 -8.601309297738348e7

In [13]:
# Get the state (km and km/s) of Mars at `et` w.r.t. Earth
spkezr("mars_barycenter", et, "J2000", "none", "earth")

([-9.561252750118901e7, -2.045743839170729e8, -8.601309297738348e7, 33.8739177120663, 3.446490512117478, 0.7085588257125508], 806.0297773638883)

In [14]:
rv_sun_eci, = spkezr("Sun",et,"J2000","NONE","Earth") # [km] and [km/s] position and velocity of the Sun in the ECI frame

([1.0923569420120437e8, -9.09859747510202e7, -3.944269164020949e7, 20.51341475783227, 20.32949234880334, 8.813008155364525], 492.1241743294098)